# Models from Bower et al. (2024)

## Data generation

This section generates and exports the output.

In [ ]:
import copy
import logging

import numpy as np
from pathlib import Path

from atmodeller import debug_logger, debug_file_logger
from atmodeller.constraints import IronWustiteBufferConstraintHirschmann, MassConstraint, SystemConstraints
from atmodeller.core import GasSpecies, Species
from atmodeller.interior_atmosphere import Planet, InteriorAtmosphereSystem
from atmodeller.initial_solution import InitialSolutionRegressor, InitialSolutionSwitchRegressor, InitialSolutionDict, InitialSolutionConstant
from atmodeller.plot import Plotter
from atmodeller.utilities import earth_oceans_to_kg

logger = debug_file_logger()
# logger.setLevel(logging.INFO)

Fixed parameters

In [ ]:
surface_temperature = 1800
number_of_realisations = 100

File paths for output

In [ ]:
trappist1e_no_sols_path = Path(f"trappist1e_{surface_temperature}K_no_sols_{number_of_realisations}its")
trappist1e_with_sols_path = Path(f"trappist1e_{surface_temperature}K_with_sols_{number_of_realisations}its")

Species to consider. These are initially excluding solubility.

In [ ]:
species = Species()
species.append(GasSpecies(formula='H2O'))
species.append(GasSpecies(formula='H2'))
species.append(GasSpecies(formula='O2'))
species.append(GasSpecies(formula='CO'))
species.append(GasSpecies(formula='CO2'))
species.append(GasSpecies(formula='CH4'))
species.append(GasSpecies(formula='N2'))
species.append(GasSpecies(formula='NH3'))
species.append(GasSpecies(formula='S2'))
species.append(GasSpecies(formula='H2S'))
species.append(GasSpecies(formula='SO2'))
species.append(GasSpecies(formula='SO'))
species.append(GasSpecies(formula='Cl2'))
species

TRAPPIST-1e planet properties

Mass and radius measurements from Agol et al. 2021; Mantle mass determined assuming same proportion as Earth

In [ ]:
trappist1e = Planet(surface_temperature=surface_temperature, mantle_mass = 2.912E24, surface_radius = 5.861E6)

Earth planet properties, which are required to scale the bulk volatile inventories for Trappist-1e. Default parameters are Earth so we only need to specify the temperature.

In [ ]:
earth = Planet(surface_temperature=surface_temperature)

Bulk silicate Earth (BSE) masses of elements in kg 

References:

Hydrogen, Carbon, Nitrogen: Sakuraba et al. 2021

Sulfur: Hirschmann et al. 2016

Chlorine: Kendrick et al. 2017


In [ ]:
earth_bse = {"H":{'min': 1.852E20,'max': 1.894E21},
             "C":{'min': 1.767E20, 'max': 3.072E21},
             'S':{'min': 8.416E20, 'max': 1.052E21},
             'N':{'min': 3.493E18, 'max': 1.052E19},
             'Cl': {'min': 7.574E19, 'max': 1.431E20}}

for element, values in earth_bse.items():
    values['mean'] = np.mean((values['min'], values['max'])) # type: ignore

earth_bse

Compute the reservoir sizes for TRAPPIST 1-e, assuming the same ppmw as Earth:

In [ ]:
trappist1e_bse = copy.deepcopy(earth_bse)
mass_scale_factor = trappist1e.mantle_mass / earth.mantle_mass

for element, values in trappist1e_bse.items():
    trappist1e_bse[element] = {key: value*mass_scale_factor for key, value in values.items()}

trappist1e_bse

Now set up the main driver of the Monte Carlo (MC) approach. This establishes the ranges over which we sample certain properties.

In [ ]:
def monte_carlo(interior_atmosphere: InteriorAtmosphereSystem, bse: dict, number_of_realisations:int=100):
    """Monte Carlo driver
    
    Args:
        interior_atmosphere: An interior-atmosphere system
        bse: Dictionary of element masses, fixed to the ppmw of the bulk silicate Earth
        number_of_realisation: Number of simulations to perform
    """

    # Parameters are normally distributed between bounds.
    number_ocean_moles = np.random.uniform(0.1, 10, number_of_realisations)
    ch_ratios = np.random.uniform(0.1, 10, number_of_realisations)
    fo2_shifts = np.random.uniform(-5, 5, number_of_realisations)

    for realisation in range(number_of_realisations):

        mass_H = earth_oceans_to_kg(number_ocean_moles[realisation])
        mass_C = ch_ratios[realisation] * mass_H
        constraints = SystemConstraints([
            MassConstraint(species="H", value=mass_H),
            MassConstraint(species="C", value=mass_C),
            MassConstraint(species="N", value=bse['N']['mean']),
            MassConstraint(species="S", value=bse['S']['mean']),
            MassConstraint(species="Cl", value=bse['Cl']['mean']),
            IronWustiteBufferConstraintHirschmann(log10_shift=fo2_shifts[realisation])
        ])

        # Extra quantities to write to the output
        # For example, it's often helpful to have the constraints expressed in a more convenient
        # form for analysis and plotting.
        extra = {'fO2_shift': fo2_shifts[realisation], 'C/H ratio': ch_ratios[realisation],
            'Number of ocean moles': number_ocean_moles[realisation]}

        interior_atmosphere.solve(constraints, extra_output=extra, tol=1e-5)

### Trappist 1-e

### CHONSCl with no solubilities

If we have no initial data to begin from, start with a dictionary of initial guesses and then switch to a regressor.

In [ ]:
values = {'H2': 100.0, 'H2O': 100, 'CO2': 100, 'CO': 500, 'S2': 1}
initial_condition_start = InitialSolutionDict(values, species=species)
initial_condition = InitialSolutionSwitchRegressor(initial_condition_start, species=species, fit=True, fit_batch_size=50, partial_fit=True, partial_fit_batch_size=25)

Otherwise, if data is available then train a network and use this from the start.

In [ ]:
trappist1e_with_sols_250_its_path = Path(f"trappist1e_{surface_temperature}K_no_sols_250its")
initial_solution = InitialSolutionRegressor.from_pickle(trappist1e_with_sols_250_its_path.with_suffix(".pkl"), species=species, fit=True, fit_batch_size=200, partial_fit=False, partial_fit_batch_size=1000)

In [ ]:
trappist1e_no_sols = InteriorAtmosphereSystem(species=species, initial_solution=initial_solution, planet=trappist1e)
monte_carlo(trappist1e_no_sols, trappist1e_bse, number_of_realisations=number_of_realisations)

In [ ]:
trappist1e_no_sols.output(file_prefix=trappist1e_no_sols_path, to_pickle=True, to_excel=True)

### CHONSCl with solubilities

This will set all the solubilities (where possible) to conform to a basaltic composition.

In [ ]:
trappist1e.melt_composition = 'basalt'

The pickle file output for the model without solubilities (above) is used to inform the initial condition. But because solubilities can strongly affect the solution, we use the `species_fill` dictionary to modify some initial guesses.

In [ ]:
species_fill = {'Cl2': 1E-6, 'H2': 1, 'H2O': 1, 'S2': 1E-6, 'SO2': 1E-6, 'SO': 1E-6}
initial_solution = InitialSolutionRegressor.from_pickle(trappist1e_no_sols_path.with_suffix(".pkl"), species=species, species_fill=species_fill, fit=True, fit_batch_size=100, partial_fit=True, partial_fit_batch_size=50)

Alternatively, here we can instead use previous output to train the network and either run with a tighter tolerance and/or for more iterations

In [ ]:
trappist1e_with_sols_200_its_path = Path(f"trappist1e_{surface_temperature}K_with_sols_200its")
initial_solution = InitialSolutionRegressor.from_pickle(trappist1e_with_sols_200_its_path.with_suffix(".pkl"), species=species, fit=False, fit_batch_size=100, partial_fit=True, partial_fit_batch_size=50)

In [ ]:
trappist1e_with_sols = InteriorAtmosphereSystem(species=species, initial_solution=initial_solution, planet=trappist1e)
monte_carlo(trappist1e_with_sols, trappist1e_bse, number_of_realisations=number_of_realisations)

In [ ]:
trappist1e_with_sols.output(file_prefix=trappist1e_with_sols_path, to_pickle=True, to_excel=True)

## Data plotting

You can just run this import rather than having to return to the top of the notebook to do the required imports

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path

from atmodeller import debug_logger, debug_file_logger
from atmodeller.plot import Plotter, AxesSpec

logger = debug_logger()
# logger.setLevel(logging.INFO)

This section plots the output. This reads the data exported by the data generation section above.

### TRAPPIST 1-e with no solubilities

In [ ]:
# Below are used to set the filename of the data to plot
plot_iterations = 250 # 10000 # 250
surface_temperature = 1800
trappist1e_no_sols_path = Path(f"trappist1e_{surface_temperature}K_no_sols_{plot_iterations}its")

plotter_no_sols = Plotter.read_pickle(trappist1e_no_sols_path.with_suffix('.pkl'))

categories: tuple[str,...] = ("Oxygen fugacity", "C/H ratio", "H budget")

Major species. These are chosen based on the fact they can contribute more than 25% of the moles of the atmosphere.

In [ ]:
axes_spec_major = AxesSpec(xylim=(0, 100), ticks=[0, 25, 50, 75, 100])

major_species = {}
major_species["CO"] = axes_spec_major
major_species["CO2"] = axes_spec_major
major_species["H2"] = axes_spec_major
major_species["H2O"] = axes_spec_major
major_species["CH4"] = axes_spec_major
major_species["H2S"] = axes_spec_major

major_species_str = "_".join(major_species.keys())

for category in categories:
    category_id: str = category[0]
    # By moles
    plotter_no_sols.species_pairplot(species=major_species, mass_or_moles='moles', category=category)
    plt.savefig(f"T1e_no_sols_{major_species_str}_by_moles_{category_id}.pdf", format='pdf')
    # By mass
    plotter_no_sols.species_pairplot(species=major_species, mass_or_moles='mass', category=category)
    plt.savefig(f"T1e_no_sols_{major_species_str}_by_mass_{category_id}.pdf", format='pdf')

Elements

In [ ]:
axes_spec_major = AxesSpec(xylim=(0, 100), ticks=[0, 25, 50, 75, 100])
axes_spec_nitrogen = AxesSpec(xylim=(0, 0.3), ticks=[0, 0.1, 0.2, 0.3])
axes_spec_sulfur = AxesSpec(xylim=(0,50), ticks=[0, 10, 20, 30, 40, 50])
axes_spec_chlorine = AxesSpec(xylim=(0,6), ticks=[0,2,4,6])

elements = {}
elements["C"] = axes_spec_major
elements["H"] = axes_spec_major
elements["O"] = axes_spec_major
elements["N"] = axes_spec_nitrogen
elements["S"] = axes_spec_sulfur
elements["Cl"] = axes_spec_chlorine

elements_str = "_".join(elements)

for category in categories:
    category_id: str = category[0]
    # By moles
    ax = plotter_no_sols.species_pairplot(species=elements, mass_or_moles='moles', category=category)
    plt.savefig(f"T1e_no_sols_{elements_str}_by_moles_{category_id}.pdf", format='pdf')
    # By mass
    ax = plotter_no_sols.species_pairplot(species=elements, mass_or_moles='mass', category=category)
    plt.savefig(f"T1e_no_sols_{elements_str}_by_mass_{category_id}.pdf", format='pdf')

Minor species. These all contribute less than 25% of the atmosphere by moles.

In [ ]:
minor_species = {}
minor_species["N2"] = None
minor_species["NH3"] = None
minor_species["S2"] = None
minor_species["SO2"] = None
minor_species["SO"] = None
minor_species["Cl2"] = None

minor_species_str = "_".join(minor_species)

for category in categories:
    category_id: str = category[0]
    # By moles
    plotter_no_sols.species_pairplot(species=minor_species, mass_or_moles='moles', category=category, plot_atmosphere=False)
    plt.savefig(f"T1e_no_sols_{minor_species_str}_by_moles_{category_id}.pdf", format='pdf')
    # By mass
    plotter_no_sols.species_pairplot(species=minor_species, mass_or_moles='mass', category=category, plot_atmosphere=False)
    plt.savefig(f"T1e_no_sols_{minor_species_str}_by_mass_{category_id}.pdf", format='pdf')

Ratio plots. These are just to sanity check that without considering solubilities the ratios in the atmosphere must correspond exactly to the ratios in the interior (melt).

In [ ]:
plotter_no_sols.ratios_pairplot(['atmosphere','total'], mass_or_moles='moles')
plt.savefig(f"trappist1e_no_sols_element_ratios_by_moles.pdf", format='pdf')

In [ ]:
plotter_no_sols.ratios_pairplot(['atmosphere','total'], mass_or_moles='mass')
plt.savefig(f"trappist1e_no_sols_element_ratios_by_mass.pdf", format='pdf')

### Trappist 1-e with solubilities

In [ ]:
trappist1e_with_sols_1000_its_path = Path(f"trappist1e_{surface_temperature}K_with_sols_1000its")
plotter_with_sols = Plotter.read_pickle(trappist1e_with_sols_1000_its_path.with_suffix('.pkl'))

In [ ]:
plotter_with_sols.species_pairplot()

In [ ]:
plotter_with_sols.species_pairplot(("H2O", "H2", "CO2", "CO"))

In [ ]:
plotter_with_sols.ratios_pairplot(['atmosphere','total'])